In [1]:
import pandas as pd
import pgmpy
import numpy as np

import networkx as nx


wiki_sent = pd.read_csv('./wikipedia-biography-dataset/wikipedia-biography-dataset/valid/valid.sent',sep='12345',header=None)
wiki_nb = pd.read_csv('./wikipedia-biography-dataset/wikipedia-biography-dataset/valid/valid.nb',header=None)
wiki_box = pd.read_csv('./wikipedia-biography-dataset/wikipedia-biography-dataset/valid/valid.box',sep ='123456',header=None)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


In [4]:
#Put sentences together

pointer = 0
sent = []

for n in wiki_nb[0]:
    
    sent.append( ' '.join(wiki_sent.iloc[ pointer  :  pointer + n,0] ))
    pointer += n

sent = pd.DataFrame(sent)

In [5]:
sent

,0
0,pope michael iii of alexandria -lrb- also know...
1,hui jun is a male former table tennis player f...
2,okan Öztürk -lrb- born 30 november 1977 -rrb- ...
3,"marie stephan , -lrb- born march 14 , 1996 -rr..."
4,leonard l. martino is a former democratic memb...
5,"salome jens -lrb- born may 8 , 1935 -rrb- is a..."
6,"carl demonte crawford -lrb- born august 5 , 19..."
7,jim bob -lrb- born james neil morrison on 22 n...
8,"riddick parker -lrb- born november 20 , 1972 i..."
9,blessed osanna of cattaro t.o.s.d. -lrb- -rrb-...


In [3]:

#Parse Relations

def parse_relations(text):
    rel = {}
    text = text.split('\t')
    for token in text:
        relation = token.rsplit('_', 1)[0]

        if relation not in rel:
            rel[relation] = []

        split  = token.split(':')
        
        if len(split) > 1:
            key  , value = split[0], split[1]
            rel[relation]  = rel[relation] + [value]


    for key in rel.keys():
        rel[key] = ' '.join(rel[key])
        
    return rel

def top_relations(fields,n=100):
    
    from collections import Counter
    flat_list = [item for list(sublist.keys()) in fields for item in sublist]

    counts = Counter(flat_list)
    counts = [c for c in counts.items() if '<none>' not in c[0]]

    relations = sorted(counts,key = lambda x:-x[1])[2:n+2]
    relations = [c[0] for c in relations]
    return relations



wiki_box[0] = wiki_box[0].apply(parse_relations)
#fields = wiki_box[0].apply(lambda x: list(x.keys()))
relations = top_relations(fields)
#fields = fields.apply(lambda x: dict([ (r,1) if r in x else (r,0) for r in relations]))


In [8]:
sorted(top_relations(list(wiki_box[0])))

['allegiance',
 'alma',
 'alma_mater',
 'associated',
 'associated_acts',
 'awards',
 'background_1:group_or',
 'background_1:solo',
 'bats',
 'battles',
 'birth',
 'birth_date',
 'birth_name',
 'birth_place',
 'branch',
 'caps',
 'caption',
 'career_start',
 'children',
 'club',
 'clubnumber',
 'clubs',
 'college',
 'country',
 'currentclub',
 'death',
 'death_date',
 'death_place',
 'debutdate',
 'debutteam',
 'debutyear',
 'district',
 'education',
 'field',
 'finalteam',
 'finalyear',
 'fullname',
 'genre',
 'goals',
 'height',
 'height_ft',
 'height_in',
 'image',
 'image_size',
 'imagesize',
 'instrument',
 'known',
 'known_for',
 'label',
 'managerclubs',
 'manageryears',
 'nationalcaps',
 'nationalgoals',
 'nationality',
 'nationalteam',
 'nationalyears',
 'native_name',
 'nickname',
 'notable',
 'ntupdate',
 'number',
 'occupation',
 'office',
 'order',
 'origin',
 'other',
 'parents',
 'party',
 'pcupdate',
 'position',
 'predecessor',
 'profession',
 'rank',
 'religion',
 'r

In [6]:
wiki_box[0].iloc[0]

{'alma': '<none>',
 'article_title': 'pope michael iii of alexandria',
 'birth': '<none> <none>',
 'birth_place': 'egypt',
 'buried': 'monastery of saint macarius the great',
 'caption:<none>': '<none>',
 'consecration:<none>': '<none>',
 'death_date': '16 march 907',
 'ended': '16 march 907',
 'enthroned': '25 april 880',
 'feast_day': '16 march -lrb- 20 baramhat in the coptic calendar -rrb-',
 'image:<none>': '<none>',
 'name': 'michael iii of alexandria',
 'nationality': 'egyptian',
 'ordination:<none>': '<none>',
 'predecessor': 'shenouda i',
 'religion': 'coptic orthodox christian',
 'residence': "saint mark 's church",
 'signature:<none>': '<none>',
 'successor': 'gabriel i',
 'title': '56th pope of alexandria & patriarch of the see of st. mark',
 'type': 'pope'}

In [ ]:
y = pd.DataFrame(list(fields))


In [148]:
import pickle
pickle.dump(relations,open("relations.p",'wb'))


In [208]:
relations = top_relations(fields,100)
fields = fields.apply(lambda x: dict([ (r,1) if r in x else (r,0) for r in relations]))


In [209]:
X_df = pd.DataFrame(list(fields))
X = np.array(X_df)

In [212]:

def prob_dist(x1,x2):
    p = np.zeros((2,2))
    p[1,1] = ((x1==1) & (x2==1)).sum() + 1
    p[1,0] = ((x1==1) & (x2==0)).sum() + 1
    p[0,1] = ((x1==0) & (x2==1)).sum() + 1 
    p[0,0] = ((x1==0) & (x2==0)).sum() + 1
    
    p = p/p.sum()
    
    return p

def factor(x,y):
    
    p_xy = prob_dist(x,y)    
    p_x, p_y = x.mean(), y.mean()
    
    I = np.zeros((2,2))
    I[0,0] = p_xy[0,0] * np.log( p_xy[0,0] /  (1-p_x) / (1-p_y))
    I[0,1] = p_xy[0,1] * np.log( p_xy[0,1] /  (1-p_x) / (p_y)) 
    I[1,0] = p_xy[1,0] * np.log( p_xy[1,0] /  (p_x) / (1-p_y)) 
    I[1,1] = p_xy[1,1] * np.log( p_xy[1,1] /  (p_x) / (p_y))
    
    return I.sum(), I


#Compute all edge weights
T = []
for i in range(X.shape[1]):
    for j in range(i+1,X.shape[1]):
        m = factor(X[:,i],X[:,j])[0]
        T.append((i,j, m))
        
T  = sorted(T ,key= lambda x: -x[2])
        
#Create Maximal spanning tree
G = nx.DiGraph()

for edge in T:
    
    G.add_edge(edge[0],edge[1])
    
    try:
        list(nx.find_cycle(G, orientation='ignore'))
        G.remove_edge(edge[0],edge[1])

    except:
        pass

In [213]:

r = list(X_df.keys())
edges = []
for e in G.edges():
    edges.append((r[e[0]] , r[e[1]]))
edges

[('caps', 'goals'),
 ('caps', 'clubs'),
 ('caps', 'pcupdate'),
 ('goals', 'position'),
 ('goals', 'height'),
 ('death_date', 'death_place'),
 ('clubs', 'years'),
 ('clubs', 'fullname'),
 ('clubs', 'youthclubs'),
 ('clubs', 'currentclub'),
 ('clubs', 'nationalteam'),
 ('clubs', 'nationality'),
 ('clubs', 'managerclubs'),
 ('clubs', 'image'),
 ('nationalcaps', 'nationalgoals'),
 ('nationalcaps', 'nationalteam'),
 ('term_end', 'term_start'),
 ('nationalteam', 'nationalyears'),
 ('nationalteam', 'ntupdate'),
 ('predecessor', 'successor'),
 ('predecessor', 'term_start'),
 ('predecessor', 'title'),
 ('position', 'statlabel'),
 ('position', 'spouse'),
 ('position', 'term_start'),
 ('party', 'term_start'),
 ('party', 'term'),
 ('statlabel', 'statvalue'),
 ('height_ft', 'height_in'),
 ('height_in', 'weight_lb'),
 ('height_in', 'position'),
 ('youthclubs', 'youthyears'),
 ('office', 'term_start'),
 ('clubnumber', 'currentclub'),
 ('height', 'weight'),
 ('debutteam', 'debutyear'),
 ('debutyear', 

In [214]:

# Defining the model
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

model = BayesianModel(edges)

# Learing CPDs using Maximum Likelihood Estimators
model.fit(X_df, estimator=MaximumLikelihoodEstimator)

In [215]:
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of allegiance:
╒═══════════════╤═══════════╕
│ allegiance(0) │ 0.96382   │
├───────────────┼───────────┤
│ allegiance(1) │ 0.0361796 │
╘═══════════════╧═══════════╛
CPD of alma:
╒═════════╤═══════════╕
│ alma(0) │ 0.936373  │
├─────────┼───────────┤
│ alma(1) │ 0.0636268 │
╘═════════╧═══════════╛
CPD of alma_mater:
╒═══════════════╤═══════════╕
│ alma_mater(0) │ 0.905384  │
├───────────────┼───────────┤
│ alma_mater(1) │ 0.0946163 │
╘═══════════════╧═══════════╛
CPD of associated:
╒═══════════════╤══════════╕
│ associated(0) │ 0.977949 │
├───────────────┼──────────┤
│ associated(1) │ 0.022051 │
╘═══════════════╧══════════╛
CPD of associated_acts:
╒════════════════════╤═══════════╕
│ associated_acts(0) │ 0.956598  │
├────────────────────┼───────────┤
│ associated_acts(1) │ 0.0434018 │
╘════════════════════╧═══════════╛
CPD of awards:
╒═══════════╤═══════════╕
│ awards(0) │ 0.948827  │
├───────────┼───────────┤
│ awards(1) │ 0.0511733 │
╘═══════════╧═══════════╛
CPD of background_1:g

In [225]:

import time
start = time.time()
y_hat =  model.predict(X_df.drop('youthclubs',axis=1).iloc[:100])
print(time.time() - start)

53.88887119293213


In [227]:
from sklearn.metrics import precision_score,recall_score

In [229]:
precision_score(X_df['youthclubs'].iloc[:100] , y_hat) , recall_score(X_df['youthclubs'].iloc[:100] , y_hat)

(0.66666666666666663, 0.5)

In [230]:
X_df['youthclubs'].iloc[:100].mean()

0.04

In [231]:
y_hat.mean()

youthclubs    0.03
dtype: float64

In [176]:
X_df.drop('youthclubs',axis=1).shape, X_df.shape

((72831, 97), (72831, 98))

In [ ]:
import pandas as pd
import pgmpy
import numpy as np

import networkx as nx


wiki_sent = pd.read_csv('./wikipedia-biography-dataset/wikipedia-biography-dataset/valid/valid.sent',sep='12345',header=None)
wiki_nb = pd.read_csv('./wikipedia-biography-dataset/wikipedia-biography-dataset/valid/valid.nb',header=None)
wiki_box = pd.read_csv('./wikipedia-biography-dataset/wikipedia-biography-dataset/valid/valid.box',sep ='123456',header=None)



In [ ]:
pointer = 0
sent = []

for n in wiki_nb[0]:
    
    sent.append( ' '.join(wiki_sent.iloc[ pointer  :  pointer + n,0] ))
    pointer += n
    